# Bicycle data melbourne analysis

In [13]:
import pandas as pd
import numpy as np
import os
from collections import Counter
from tqdm import tqdm

## Data validation 

The bicycle data is delivered in a large number of CSV files, without an accompanying data dictionary.  To be sure the same columns are shared by all CSVs, such that they can be appended to a master file, we iterate over them and tally up the unique column combinations.  Ideally, the results will be a single combination of columns with a tally count the length of the number of CSV files.  Let's see!

In [ ]:
rootdir ='../data'
column_array = []
tally = pd.DataFrame(columns=["count"])
counter = 0
for subdir, dirs, files in tqdm(os.walk(rootdir)):
    for file in files:
        if ('.csv' in file) and ('.zip' not in file):
            # read in CSV
            df = pd.read_csv(os.path.join(subdir,file))
            # store list of columns in variable df_columns as a string
            df_columns = f"{df.columns.to_list()}"
            # if CSV columns string is in the tally index, increment this
            if df_columns in tally.index:
                tally[tally.index==df_columns] += 1
            # otherwise add CSV columns string to the tally index
            else:
                tally.loc[df_columns] = 1
            # increment a counter; athough theoretically this should only sum to the sum of tallys!
            counter+=1
            
print(counter)

279it [06:22,  1.26s/it]

In [ ]:
tally